In [ ]:
#pip install bs4
#pip install requests

Parsing part of project

In [4]:
from bs4 import BeautifulSoup
import requests


# Define headers for the HTTP request
HEADERS = {'User-Agent': 'Mozilla/5.0'}

# Fetch the HTML content from the Wikipedia page
response = requests.get('https://en.wikipedia.org/wiki/List_of_highest-grossing_films', headers=HEADERS)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "lxml")

# Find the table containing the film data
table = soup.find('table')

# Initialize a list to store the parsed film data
films = []

# Iterate over each row in the table (excluding the header row)
for row in table.select('tr')[1:]:
    film = {}

    # Handle special cases for 'RK' and 'TS3'
    if row.find('a').text in ['RK', 'TS3']:
        film['name'] = row.find_all('a')[1].text
        film['href'] = row.find_all('a')[1]['href']
    else:
        film['name'] = row.find('a').text
        film['href'] = row.find('a')['href']

    # Extract box office revenue and year information
    film['Box Office Revenue'] = row.find_all('td')[2].text[:-1].replace(',', '')
    film['Box Office Revenue'] = film['Box Office Revenue'] [film['Box Office Revenue'].find('$'):]
    
    film['year'] = row.find_all('td')[3].text.strip()[:-1]

    # Append the film dictionary to the films list
    films.append(film)

# Iterate over each film to fetch additional details like director and country
for film in films:
    url = "https://en.wikipedia.org" + film['href']
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "lxml")

    # Find the 'Directed by' and 'Country' sections
    author = soup.find('th', text='Directed by')
    country = soup.find('th', text='Country') or soup.find('th', text='Countries')

    # Extract director(s) and country(s)
    for tr in soup.find_all('tr'):
        if author in tr:
            if tr.find('td').find('ul') is None:
                film['Director'] = tr.find('td').find('a').text
            else:
                film['Director'] = [li.text for li in tr.find('td').find('ul').find_all('li')]
        
        if country in tr:
            if tr.find('td').find('ul') is None:
                film['Country'] = tr.find('td').text.strip()
            else:
                film['Country'] = [li.text[:-3] for li in tr.find('td').find('ul').find_all('li')]
            break
films

[{'name': 'Avatar',
  'href': '/wiki/Avatar_(2009_film)',
  'Box Office Revenue': '$2923706026',
  'year': '200',
  'Director': 'James Cameron',
  'Country': ['United Kingdom', 'United States']},
 {'name': 'Avengers: Endgame',
  'href': '/wiki/Avengers:_Endgame',
  'Box Office Revenue': '$2797501328',
  'year': '201',
  'Director': ['Anthony Russo', 'Joe Russo'],
  'Country': 'United States'},
 {'name': 'Avatar: The Way of Water',
  'href': '/wiki/Avatar:_The_Way_of_Water',
  'Box Office Revenue': '$2320250281',
  'year': '202',
  'Director': 'James Cameron',
  'Country': 'United States'},
 {'name': 'Titanic',
  'href': '/wiki/Titanic_(1997_film)',
  'Box Office Revenue': '$2257844554',
  'year': '199',
  'Director': 'James Cameron',
  'Country': 'United States'},
 {'name': 'Star Wars: The Force Awakens',
  'href': '/wiki/Star_Wars:_The_Force_Awakens',
  'Box Office Revenue': '$2068223624',
  'year': '201',
  'Director': 'J. J. Abrams',
  'Country': 'United States'},
 {'name': 'Avenger

Database part

In [5]:
!pip install sqlalchemy

In [6]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json


In [7]:
# Create an in-memory SQLite database engine
engine = create_engine('sqlite+pysqlite:///:memory:', echo=True)

In [8]:
from sqlalchemy.ext.declarative import declarative_base

# Define the base class for declarative models
Base = declarative_base()

C:\Users\Nikita\AppData\Local\Temp\ipykernel_4996\3575547011.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [9]:
from sqlalchemy import Column, Integer, String


# Define a model class for the films table
class Films(Base):
    __tablename__ = 'films'
    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String, nullable=False)
    release_year = Column(Integer)
    director = Column(String)
    box_office = Column(String)
    country = Column(String)


In [10]:
# Create all tables in the database
Base.metadata.create_all(engine)

2025-02-25 23:12:27,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 23:12:27,875 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("films")
2025-02-25 23:12:27,876 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-25 23:12:27,878 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("films")
2025-02-25 23:12:27,879 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-25 23:12:27,882 INFO sqlalchemy.engine.Engine 
CREATE TABLE films (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	release_year INTEGER, 
	director VARCHAR, 
	box_office VARCHAR, 
	country VARCHAR, 
	PRIMARY KEY (id)
)


2025-02-25 23:12:27,884 INFO sqlalchemy.engine.Engine [no key 0.00153s] ()
2025-02-25 23:12:27,886 INFO sqlalchemy.engine.Engine COMMIT


In [11]:

# Function to preprocess the raw film data
def preprocess(films_raw):
    films = []

    for film_raw in films_raw:
        film = {
            'title': film_raw['name'],
            'release_year': int(film_raw['year']),
            'director': ', '.join(director for director in film_raw['Director']) if isinstance(film_raw['Director'], list) else film_raw['Director'],
            'box_office': film_raw['Box Office Revenue'][1:],
            'country': ', '.join(country for country in film_raw['Country']) if isinstance(film_raw['Country'], list) else film_raw['Country']
        }
        films.append(film)

    return films

In [12]:
Session = sessionmaker(bind=engine)
session = Session()

# Clear existing records in the films table
session.query(Films).delete()

# Preprocess the film data and add it to the database
films_rows = preprocess(films)
films_rows = [Films(**f) for f in films_rows]
session.add_all(films_rows)
session.commit()

2025-02-25 23:12:32,931 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 23:12:32,934 INFO sqlalchemy.engine.Engine DELETE FROM films
2025-02-25 23:12:32,935 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ()
2025-02-25 23:12:32,940 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?) RETURNING id
2025-02-25 23:12:32,941 INFO sqlalchemy.engine.Engine [generated in 0.00023s (insertmanyvalues) 1/50 (ordered; batch not supported)] ('Avatar', 200, 'James Cameron', '2923706026', 'United Kingdom, United States')
2025-02-25 23:12:32,943 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?) RETURNING id
2025-02-25 23:12:32,944 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/50 (ordered; batch not supported)] ('Avengers: Endgame', 201, 'Anthony Russo, Joe Russo', '2797501328', 'United States')
2025-02-25 23:12:32,945 INFO sqlalchemy.engine

In [13]:
# Query the films table and prepare the result for JSON output
films = session.query(Films)

In [14]:
result = {'films': []}

for f in films:
    f_dict = f.__dict__
    f_dict.pop('_sa_instance_state', None)  # Remove the internal SQLAlchemy state object
    result['films'].append({
        'director': f_dict['director'],
        'country': f_dict['country'],
        'release_year': f_dict['release_year'],
        'box_office': f_dict['box_office'],
        'title': f_dict['title']
    })
result

2025-02-25 23:12:37,610 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 23:12:37,612 INFO sqlalchemy.engine.Engine SELECT films.id AS films_id, films.title AS films_title, films.release_year AS films_release_year, films.director AS films_director, films.box_office AS films_box_office, films.country AS films_country 
FROM films
2025-02-25 23:12:37,613 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ()


{'films': [{'director': 'James Cameron',
   'country': 'United Kingdom, United States',
   'release_year': 200,
   'box_office': '2923706026',
   'title': 'Avatar'},
  {'director': 'Anthony Russo, Joe Russo',
   'country': 'United States',
   'release_year': 201,
   'box_office': '2797501328',
   'title': 'Avengers: Endgame'},
  {'director': 'James Cameron',
   'country': 'United States',
   'release_year': 202,
   'box_office': '2320250281',
   'title': 'Avatar: The Way of Water'},
  {'director': 'James Cameron',
   'country': 'United States',
   'release_year': 199,
   'box_office': '2257844554',
   'title': 'Titanic'},
  {'director': 'J. J. Abrams',
   'country': 'United States',
   'release_year': 201,
   'box_office': '2068223624',
   'title': 'Star Wars: The Force Awakens'},
  {'director': 'Anthony Russo, Joe Russo',
   'country': 'United States',
   'release_year': 201,
   'box_office': '2048359754',
   'title': 'Avengers: Infinity War'},
  {'director': 'Jon Watts',
   'country'

In [15]:
with open('filmsFinal.json', 'w') as f:
    json.dump(result, f)

# Close the session
session.close()

2025-02-25 23:13:16,056 INFO sqlalchemy.engine.Engine ROLLBACK
